In [19]:
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import types
types.RecognitionConfig.AudioEncoding.FLAC
from pydub import AudioSegment
import moviepy.editor as mp
from moviepy.editor import *
from ffprobe import FFProbe
import os
from os import path
import argparse
import wave
import re
import ffmpeg
import sys
import io

#declaring path of folder
path='/Users/iqra/Desktop/folder'


In [20]:
#function to transcribe filr
def transcribe_file(speech_file):
    
    #initialize GCP credentials and create client objecy
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/iqra/Downloads/hollohk-308819-7ba32dd5db3b.json"
    client = speech.SpeechClient()

    # reading audio file
    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()
        
    #extracting frame rate of file   
    with wave.open(speech_file, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        
    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="en-US",
        enable_word_time_offsets=True,
        sample_rate_hertz= frame_rate,
        enable_speaker_diarization=True,
        diarization_speaker_count=2,
    )
   

    # start speech_python_migration_sync_response
    response = client.recognize(config=config, audio=audio)
    result = response.results[-1]

    words_info = result.alternatives[0].words

    # Printing out the output:
    tag=1
    speaker=""

    for word_info in words_info:
        if word_info.speaker_tag==tag:
            speaker=speaker+" "+word_info.word

        else:
            print("speaker {}: {}".format(tag,speaker))
            tag=word_info.speaker_tag
            speaker=""+word_info.word
    
    


    for result in response.results:
            
        best_alternative = result.alternatives[0]
        confidence = best_alternative.confidence
        print("*" * 20)
        print(f"Confidence: {confidence:.0%}")
        print("*" * 20)
        break
        
    print("-" * 80)
        
        
        ##uncomment out the next section to get word timestsamps
    """
        for word_info in best_alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time

            print(
                f"Word: {word}, start_time: {start_time.total_seconds()}, end_time: {end_time.total_seconds()}"
            )
    """

    # [END speech_python_migration_sync_response]


# end function

In [21]:
#function to convert stereo audio to mono
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

In [22]:
def mp4_to_mp3(filepath):
    
        video = VideoFileClip(filepath)
        list_filepath = list(filepath)
        if list_filepath[-1] == '4':
            list_filepath[-1] = '3'
            filepath = ''.join(list_filepath)
            audio = video.audio
            audio.write_audiofile(filepath, logger=None)



In [23]:
#state ffmpeg path
AudioSegment.converter = '/usr/local/bin/ffmpeg'


def main():

    #Change working directory
    os.chdir(path)

    audio_files = os.listdir()

    count=1
    for file in audio_files:
    #spliting the file into the name and the extension
        name, ext = os.path.splitext(file)
        if ext == ".mp4":
            filepath=path+"/"+file
            mp4_to_mp3(filepath)
            
    audio_files = os.listdir()

        
    for file in audio_files:
    #spliting the file into the name and the extension
        name, ext = os.path.splitext(file)
        if ext == ".mp3":
            mp3_sound = AudioSegment.from_mp3(file)
           #rename them using the old name + ".wav"
            mp3_sound.export("{0}.wav".format(name), format="wav")
          
            
            
    audio_files = os.listdir()

    
    for file in audio_files:
    #spliting the file into the name and the extension
        name, ext = os.path.splitext(file)
        if ext == ".wav":
            stereo_to_mono(file)
            transcribe_file(file)



main()

speaker 1: 
speaker 2: to be honest answer the ultimate dream is to good to be working on something of my own which which I can say that.
********************
Confidence: 89%
********************
--------------------------------------------------------------------------------
speaker 1:  let me see if they can provide you some Brokers number it'll definitely help you further how do you commute everyday sometimes and sometimes by Metro
speaker 2: I cash that must be
speaker 1: pretty hectic for you isn't it it
********************
Confidence: 93%
********************
--------------------------------------------------------------------------------
